## In this notebook, we'll try to get to know about our H&M dataset.
> 📝 **Note:** The EDA strategies are adopted from my instructor. Because of my limited knowledge, feel free to add more !

## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import sys

import datasets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datasets import load_dataset
from evidently.metric_preset import DataQualityPreset
from evidently.report import Report
from plotly.subplots import make_subplots
from pydantic import BaseModel

datasets.logging.set_verbosity_error()

sys.path.insert(0, "..")

from src.utils.data_prep import parse_dt

# Set up plot styling
from src.visualization.setup import color_scheme

## Configuration

In [3]:
from src.config.dataset_config import dataset_config

class Args(BaseModel):
    hf_dataset_path: str = dataset_config.HF_DATASET_PATH
    hf_dataset_token: str = dataset_config.HF_TOKEN
    report_sample_num_rows: int = 10000  #sampling data in order to simulate real distribution
    random_seed: int = 41


args = Args()

2024-12-09 20:49:54.565 | INFO     | src.config.dataset_config:<module>:29 - Dataset Path: dinhlnd1610/HM-Personalized-Fashion-Recommendations


## Load data

In [4]:
article_metadata = load_dataset(
    args.hf_dataset_path,
    token= args.hf_dataset_token,
    name="articles",
    split="train",
    trust_remote_code=True,
)

In [5]:
article_metadata[0]

{'article_id': 108775015,
 'product_code': 108775,
 'prod_name': 'Strap top',
 'product_type_no': 253,
 'product_type_name': 'Vest top',
 'product_group_name': 'Garment Upper body',
 'graphical_appearance_no': 1010016,
 'graphical_appearance_name': 'Solid',
 'colour_group_code': 9,
 'colour_group_name': 'Black',
 'perceived_colour_value_id': 4,
 'perceived_colour_value_name': 'Dark',
 'perceived_colour_master_id': 5,
 'perceived_colour_master_name': 'Black',
 'department_no': 1676,
 'department_name': 'Jersey Basic',
 'index_code': 'A',
 'index_name': 'Ladieswear',
 'index_group_no': 1,
 'index_group_name': 'Ladieswear',
 'section_no': 16,
 'section_name': 'Womens Everyday Basics',
 'garment_group_no': 1002,
 'garment_group_name': 'Jersey Basic',
 'detail_desc': 'Jersey top with narrow shoulder straps.'}

- Small experiment : See the impact of num_proc

In [6]:
%%time

transactions = load_dataset(
    args.hf_dataset_path,
    token= args.hf_dataset_token,
    name="transactions",
    split="train",
    trust_remote_code=True,
    num_proc = 8
)

CPU times: user 542 ms, sys: 856 ms, total: 1.4 s
Wall time: 5.67 s


In [7]:
%%time

transactions = load_dataset(
    args.hf_dataset_path,
    token= args.hf_dataset_token,
    name="transactions",
    split="train",
    trust_remote_code=True,
    num_proc = 1
)

CPU times: user 526 ms, sys: 312 ms, total: 838 ms
Wall time: 3.83 s


In [8]:
transactions

Dataset({
    features: ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'],
    num_rows: 31788324
})

In [9]:
transactions[0]

{'t_dat': '2018-09-20',
 'customer_id': '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
 'article_id': 663713001,
 'price': 0.0508305084745762,
 'sales_channel_id': 2}

In [10]:
customer_metadata = load_dataset(
    args.hf_dataset_path,
    token= args.hf_dataset_token,
    name="customers",
    split="train",
    trust_remote_code=True,
    num_proc = 8
)

In [11]:
customer_metadata[0]

{'customer_id': '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
 'FN': None,
 'Active': None,
 'club_member_status': 'ACTIVE',
 'fashion_news_frequency': 'NONE',
 'age': 49.0,
 'postal_code': '52043ee2162cf5aa7ee79974281641c6f11a68d276429a91f8ca0d4b6efa8100'}

In [12]:
train_raw_df = transactions.to_pandas()

In [13]:
train_raw_df.head(5)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [14]:
article_metadata_raw_df = article_metadata.to_pandas()

In [15]:
article_metadata_raw_df.head(5)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [16]:
# Convert timpestamp column (series) to pandas date-time format
train_df = train_raw_df.pipe(parse_dt)

In [17]:
train_df.head(5)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


## Basic data quality report with Eviently AI

In [18]:
# Sampling data without replacing original data
train_sample_df = train_df.sample(
    args.report_sample_num_rows, replace=False, random_state=args.random_seed
)

In [19]:
data_quality_report = Report(
    metrics=[
        DataQualityPreset(),
    ],
    options=[color_scheme],
)

data_quality_report.run(reference_data=None, current_data=train_sample_df)
data_quality_report.save_html("./data_quality_report.html")

## Distribution per time

In [20]:
def plot_interaction_by_dayofweek(df):
    plot_df = df.assign(
        day_of_week=df["t_dat"]
        .dt.day_name()                 # Get the day in week
    )

    interaction_count_by_day = (
        plot_df.groupby("day_of_week")
        .size()
        .reindex(
            [
                "Monday",
                "Tuesday",
                "Wednesday",
                "Thursday",
                "Friday",
                "Saturday",
                "Sunday",
            ]
        )
    )

    # Create the plot
    fig = px.bar(
        x=interaction_count_by_day.index,
        y=interaction_count_by_day.values,
        labels={"x": "Day of the Week", "y": "Number of Interactions"},
        title=f"Interaction Count by Day of the Week",
        text=[f"{val:,.0f}" for val in interaction_count_by_day.values],
        height=500,
        width=700,
    )

    fig.update_layout()

    fig.show()

plot_interaction_by_dayofweek(train_sample_df)

In [21]:
def plot_interaction_by_month(df):
    # Extract the month from the timestamp
    plot_df = df.assign(
        month_of_year=df["t_dat"]
        .dt.month
    )

    # Group by the month and count the number of interactions
    interaction_count_by_month = (
        plot_df.groupby("month_of_year").size().reindex(range(1, 13), fill_value=0)
    )

    # Create the plot
    fig = px.bar(
        x=interaction_count_by_month.index,
        y=interaction_count_by_month.values,
        labels={"x": "Month", "y": "Number of Interactions"},
        title=f"Interaction Count by Month",
        text=[f"{val:,.0f}" for val in interaction_count_by_month.values],
        height=500,
        width=1200,
    )

    fig.update_layout(
        xaxis_tickmode="linear",
        xaxis_tickvals=interaction_count_by_month.index,
        xaxis_ticktext=[
            pd.to_datetime(month, format="%m").strftime("%B")
            for month in interaction_count_by_month.index
        ],
    )

    fig.show()


plot_interaction_by_month(train_sample_df)

## Sparsity

In [22]:
def calculate_sparsity(df, user_col="customer_id", item_col="article_id"):
    return 1 - df.shape[0] / (df[user_col].nunique() * df[item_col].nunique())


print(f"Sparsity: {calculate_sparsity(train_df):,.4%}")

Sparsity: 99.9777%


## User-Item distribution

In [23]:
def plot_user_rating_distribution(df, col = "customer_id", width = 12000):
    user_rating_counts = df.groupby(col).size().reset_index(name = "rating_count")

    fig = px.histogram(
        user_rating_counts,
        x = "rating_count",
        labels={"rating_count": "Number of Buying"},
        title=f"Number of Ratings per {col}",
        text_auto=True,
        height=500,
        width=width,
    )

    fig.update_yaxes(title_text=f"Number of {col}")

    fig.show()

plot_user_rating_distribution(train_sample_df, col="customer_id", width=None)
plot_user_rating_distribution(train_sample_df, col="article_id", width=None)

## Most popular items

In [62]:
popular_items_df = (
    train_sample_df.groupby(["article_id"], as_index = False)      # do not create article_id as index, so when .size(), we can obtain DataFrame
    .agg(
    size = ("customer_id" , "size"),
    price = ("price", lambda x: list(x)))
    .pipe(lambda df : pd.merge(df, article_metadata_raw_df, how = "left", on = "article_id"))
    .sort_values("size", ascending = False)[["article_id", "product_type_name", "prod_name", "price", "size"]]
    .head(5)
)

with pd.option_context("display.max_colwidth", None):
    display(popular_items_df)

,article_id,product_type_name,prod_name,price,size
3429,706016001,Trousers,Jade HW Skinny Denim TRS,"[0.0314237288135593, 0.03, 0.0243898305084745, 0.0329491525423728, 0.0271016949152542, 0.0338813559322033, 0.0338813559322033, 0.0311694915254237, 0.0271016949152542, 0.0241525423728813, 0.0322033898305084, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0271016949152542, 0.0338813559322033]",21
165,372860002,Socks,7p Basic Shaftless,"[0.0118474576271186, 0.0135423728813559, 0.0135423728813559, 0.0135423728813559, 0.0135423728813559, 0.0135423728813559, 0.0135423728813559, 0.0135423728813559, 0.0135423728813559, 0.0116101694915254, 0.0135423728813559]",11
3430,706016002,Trousers,Jade HW Skinny Denim TRS,"[0.0335423728813559, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033]",10
4994,759871002,Vest top,Tilda tank,"[0.0050677966101694, 0.0047966101694915, 0.0050677966101694, 0.0067627118644067, 0.0067627118644067, 0.0050677966101694, 0.0067627118644067, 0.0047966101694915, 0.0054067796610169]",9
723,554450001,Trousers,Julia RW Skinny Denim TRS,"[0.0304915254237288, 0.0326949152542372, 0.0338813559322033, 0.0304915254237288, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033, 0.0338813559322033]",9


- By chance, at here, we can see that there are some similar product_name sharing different product_id. Let's find out why

In [88]:
article_metadata_raw_df[article_metadata_raw_df["prod_name"] == "Jade HW Skinny Denim TRS"]

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
53892,706016001,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,9,Black,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53893,706016002,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,71,Light Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53894,706016003,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,73,Dark Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53895,706016004,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010014,Placement print,71,Light Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53896,706016006,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,73,Dark Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53897,706016007,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,73,Dark Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53902,706016015,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,8,Dark Grey,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53904,706016019,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,73,Dark Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53908,706016028,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,73,Dark Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...
53909,706016029,706016,Jade HW Skinny Denim TRS,272,Trousers,Garment Lower body,1010016,Solid,71,Light Blue,...,Trousers,D,Divided,2,Divided,53,Divided Collection,1009,Trousers,High-waisted jeans in washed superstretch deni...


In [63]:
# Bar chart for popular items
grouped_df = popular_items_df.groupby("product_type_name", as_index=False).agg(total_size=("size", "sum"))

# Bar chart
fig = px.bar(
    grouped_df,
    x="product_type_name",
    y="total_size",
    color="product_type_name",
    title="Total Size by Product Type",
    labels={"product_type_name": "Product Type", "total_size": "Bought"},
)

fig.show()

## Item popularity drift over time

In [75]:
popular_items_data_df = train_df.loc[
    lambda df: df["article_id"].isin(popular_items_df["article_id"].values)
].pipe(lambda df: pd.merge(df, popular_items_df, on="article_id", how="inner")).drop(["price_y", "size"], axis=1)

popular_items_data_df

,t_dat,customer_id,article_id,price_x,sales_channel_id,product_type_name,prod_name
0,2018-09-20,02f6cba4ca8942ae995a5723a271ac967a3b51973fef55...,554450001,0.033881,2,Trousers,Julia RW Skinny Denim TRS
1,2018-09-20,13e31ca3e1e8cee4f5e995ddcd8bc9bb3e012b42da9737...,554450001,0.030492,2,Trousers,Julia RW Skinny Denim TRS
2,2018-09-20,1c3ba7ee0819e0d7a8abae331fce61ad8198be0bc3e140...,554450001,0.027102,2,Trousers,Julia RW Skinny Denim TRS
3,2018-09-20,217977ee5ca3f9c5f59282e6742005ac6703c8a4e69513...,372860002,0.013542,1,Socks,7p Basic Shaftless
4,2018-09-20,217bd772e946a013afd4482a4faa7bd1949c1b4b9a0632...,554450001,0.033881,2,Trousers,Julia RW Skinny Denim TRS
...,...,...,...,...,...,...,...
151357,2020-09-22,fccf98f842fab66d98dcd454f2ab65bb3ddd789cc90d62...,706016001,0.033881,2,Trousers,Jade HW Skinny Denim TRS
151358,2020-09-22,fdc53a7ef3a228d6101c8463d2c2bd2ed5eb126ec0461f...,706016001,0.033881,2,Trousers,Jade HW Skinny Denim TRS
151359,2020-09-22,fe376f1ea62de3dafb2cbc503127362efed9015ffe2ed6...,706016001,0.033881,2,Trousers,Jade HW Skinny Denim TRS
151360,2020-09-22,fe376f1ea62de3dafb2cbc503127362efed9015ffe2ed6...,706016001,0.033881,2,Trousers,Jade HW Skinny Denim TRS


In [83]:
def plot_popular_item_popularity_distribution(df: pd.DataFrame, item_col = "article_id", timestamp_col = "t_dat"):

    # Group by "article_id" and timestamp by month
    plot_df = (
        df.groupby([df[item_col], df[timestamp_col].dt.to_period("M")])
        .size()
        .reset_index(name = "count")
    )

    plot_df[timestamp_col] = plot_df[timestamp_col].dt.to_timestamp()

    # Get unique article_id for plotting sub-charts
    unique_items = plot_df[item_col].unique()

    # Determine the number of rows needed
    num_cols = 3
    num_rows = math.ceil(len(unique_items) / num_cols)

    print(unique_items)

plot_popular_item_popularity_distribution(popular_items_data_df)

[372860002 554450001 706016001 706016002 759871002]
